### DeepSORT
- SORT는 state estimation 불확실성이 낮을 때만 associate metric가 정확했기 때문에, occlusion을 잘 처리하지 못하였음. 
- 이를 해결하기 위해 SORT 알고리즘에 __(Motion &) Appearance information__을 추가한 모델
- Longer periods of occlusions와 Identity switches 문제를 해결할 수 있었음
- SORT에 사용된 recursive Kalman filtering과 frame-by-frame data association을 동일하게 사용하고 있음  

### 1. Track Handling & State Estimation
- Track Handling과 Kalman filtering은 SORT를 그대로 사용하고 있으나, state space가 약간 다름
- 다음 프레임의 aspect ratio가 동일할 것이라고 가정한 SORT와 달리, DeepSORT는 8개의 차원을 사용하여 state space를 사용
<img src='img/deepsort_1.png' width='150'>
> - (u, v) : bounding box center position
> - $\gamma$ : aspect ratio
> - h : height

### 2. Assignment Problem
- motion information : short-term predictions에 유용
- appearance information은 long-term metrics에 유용
- 두 정보의 가중합으로 cost metrics를 정의 (하지만, experiment를 해보니 $\lambda=0$일 때가 성능이 좋았음. 즉, appearance information만 사용)
<img src='img/deepsort_4.png' width='300'>
<img src='img/deepsort_5.png' width='120'>

#### 1) Motion information
- predicted Kalman states와 newly arrived measurements의 Mahalanobis 거리를 사용
- 역감마분포의 95% 신뢰수준에 해당하는 값을 threshold로 지정하여 가능성이 적은 association을 제거
<img src='img/deepsort_2.png' width='250'>
<img src='img/deepsort_3.png' width='200'>

#### 2) Appearance information
- i번째 track과 j번째 detection의 cosine distance를 사용
- appearance vector를 구하기 위해서 pre-trained CNN를 사용  
<img src='img/deepsort_6.png' width='300'>
<img src='img/deepsort_7.png' width='120'>
<img src='img/deepsort_8.png' width='300'>
<center>pre-trained CNN architecture</center>

> __cf) Mahalanobis distance & Cosine distance__   

> <마할라노비스 거리(Mahalanobis distance)>
- 마할라노비스 거리는 확률분포를 고려한 거리를 의미하여, 공식은 아래와 같다.  
<img src='img/deepsort_10.png' width='250'>    


> - 이때, S는 공분산 행렬을 의미하며, 공분산 행렬이 단위행렬인 경우, 유클리디안 거리와 같아진다.
> - 좀더 직관적인 이해를 위해 예시를 들어 설명하고자 한다. 아래의 그림에서의 유클리디안 거리는 A-C가 A-B보다 긴 것을 확인할 수 있다. 하지만, 확률 분포를 생각해보면 C가 B에 비해서 A에 더 가까이 있을 확률이 크기 때문에 거리도 더 가깝다고 할 수 있다.  
<img src='img/deepsort_11.png' width='220'>  


> - 그렇다면 실제 마할라노비스 거리를 구해보자. 우선, 마할라노비스 거리를 구하기 위해 화이트닝 변환을 해주어야 한다. 이때, 화이트닝 변환이란 공분산 행렬이 단위 행렬이 되도록 벡터 x를 y로 변환하는 것을 의미하며, 변환 공식은 아래와 같다.  
<img src='img/deepsort_12.png' width='150'>


> - 위의 점들을 화이트닝을 변환을 해주면, 아래의 그림과 같아진다(파란 점들은 생략하였다). 공분산 행렬이 단위행렬이 되었기때문에 마할라노비스 거리는 유클리디안 거리와 같아지고, A-C의 거리가 A-B보다 더 짧아진 것을 확인할 수 있다.
<img src='img/deepsort_13.png' width='100'>


> <코사인 거리(Cosine Distance)>

> - 코사인 거리는 두 벡터의 크기에 의존하지 않으며, 각도만 고려한 거리이며, 공식은 아래와 같다.  
<img src='img/deepsort_14.png' width='250'>  


> - 이때, Cosine Similiarity는 아래와 같이 정의된다.   
<img src='img/deepsort_15.png' width='300'>

> - 참고
    - http://asq.kr/XnmkEk
    - https://velog.io/@poiu8944/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9DDistance-Metric-Types
    - https://techblog-history-younghunjo1.tistory.com/84

### 3. Matching Cascade
- 오랜 기간동안 occlusion이 일어나면 Kalman filter의 prediction이 uncertainty를 증가시킨다. 하지만, 직관적이지 않게도 Mahalanobis distance를 예시처럼 uncertainty를 선호한다. 
- 이를 해결하기 위해 __더 적은 age를 갖는 track에게 우선순위를 부여__하는 알고리즘을 사용하였다. 전체 알고리즘은 아래와 같다.  
<img src='img/deepsort_9.png' width='400'>

### 4. Experiements
- SORT보다 __ID switches__를 45% 가량 줄였으며, 다른 online methods 중 가장 적은 ID switches를 나타낸다
- 또한, longer occlusions에도 ID를 잘 유지할 수 있었다.
- 하지만, __FP__의 수가 너무 많았다.  
<img src='img/deepsort_16.png' width='600'>

### 5. Conclustion

- 본 논문에서는 SORT에 appearance information을 추가한 모델을 제시하였으며, 이로 인해 longer periods의 occlusion에도 잘 추적할 수 있었다. 
- 그럼에도 불구하고, 알고리즘이 단순하고 real-time으로 사용할 수 있으므로 현재까지 많이 사용되는 모델이다.

## Code
https://github.com/ZQPei/deep_sort_pytorch
### yolov3_deepsort.py

In [ ]:
import os
import cv2
import time
import argparse
import torch
import warnings
import numpy as np

from detector import build_detector
from deep_sort import build_tracker
from utils.draw import draw_boxes
from utils.parser import get_config
from utils.log import get_logger
from utils.io import write_results

In [ ]:
class VideoTracker(object):
    def __init__(self, cfg, args, video_path):
        self.cfg = cfg
        self.args = args
        self.video_path = video_path
        self.logger = get_logger("root")

        # gpu 사용여부
        use_cuda = args.use_cuda and torch.cuda.is_available()
        if not use_cuda:
            warnings.warn("Running in cpu mode which maybe very slow!", UserWarning)

        # args.display이면 창을 띄워줌
        if args.display:
            cv2.namedWindow("test", cv2.WINDOW_NORMAL)  
                # cv2.namedWindow : 첫번째 인자(winname)라는 창을 만들어줌
                # 두번째 인자(flag) : 창 옵션 
                #   - cv2.WINDOW_NORMAL : 사용자가 창 크기 조장가능
                #   - cv2.WINDOW_AUTOSIZE : 이미지와 동일한 크기로 크기 조정불가
            cv2.resizeWindow("test", args.display_width, args.display_height)
                # cv2.resizeWindow : winname창의 크기를 (width, height)크기로 조정해줌

        # 카메라 열기
        # -1이면 VideoCapture 객체만 생성, -1이 아니면 해당 웹캠 열기
        # (웹캠이 하나면 0, 2개 이상이면 첫 웹캠은 0, 두번째 웹캠은 1로 지정됨)
        if args.cam != -1:
            print("Using webcam " + str(args.cam))
            self.vdo = cv2.VideoCapture(args.cam)
        else:
            self.vdo = cv2.VideoCapture()

        # detector, deepsort모델 build해주기
        self.detector = build_detector(cfg, use_cuda=use_cuda) # YOLO v3 모델
        self.deepsort = build_tracker(cfg, use_cuda=use_cuda)  # DeepSORT 모델
        self.class_names = self.detector.class_names

    def __enter__(self):
        '''
        __enter__ : with구문에 진입하는 시점에 자동으로 호출하는 메서드
        '''

        # args.cam이 -1이 아닌 경우, 프레임 받아오기
        if self.args.cam != -1:
            ret, frame = self.vdo.read()  
                # 비디오의 한 프레임을 읽음
                # return 값
                #   - ret : 제대로 프레임을 읽으면 T, 실패하면 F
                #   - fram : 읽은 프레임이 나옴

            # 프레임을 제대로 읽지 못하면 Assertionerror 발생
            assert ret, "Error: Camera error"  

            # 프레임의 width, height 가져오기
            self.im_width = frame.shape[0]
            self.im_height = frame.shape[1]

        # args.cam이 -1인 경우, video_path의 video open해주기
        else:
            assert os.path.isfile(self.video_path), "Path error"
            self.vdo.open(self.video_path)
            self.im_width = int(self.vdo.get(cv2.CAP_PROP_FRAME_WIDTH))
            self.im_height = int(self.vdo.get(cv2.CAP_PROP_FRAME_HEIGHT))
                # get : 항목 확인, set : 항목 변경
                # cv2.CAP_PROP_FRAME_WIDTH : 프레임의 폭
                # cv2.CAP_PROP_FRAME_HEIGHT : 프레임의 높이
            assert self.vdo.isOpened()

        # 결과를 저장해줄 dirs 만들어주기
        if self.args.save_path:
            os.makedirs(self.args.save_path, exist_ok=True)

            # path of saved video and results
            self.save_video_path = os.path.join(self.args.save_path, "results.avi")
            self.save_results_path = os.path.join(self.args.save_path, "results.txt")

            # create video writer
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            self.writer = cv2.VideoWriter(self.save_video_path, fourcc, 20, (self.im_width, self.im_height))

            # logging
            self.logger.info("Save results to {}".format(self.args.save_path))

        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        '''__exit__ : with 구문을 빠져나오기 직전에 호출되는 메소드'''
        if exc_type:
            print(exc_type, exc_value, exc_traceback)

    def run(self):
        results = []
        idx_frame = 0

        while self.vdo.grab():  # grab : 카메라 장치에 다음 프레임을 획득하라는 명령내리는 함수
            idx_frame += 1
            if idx_frame % self.args.frame_interval:  # frame_interval을 둬서 처리하기
                continue

            start = time.time()
            _, ori_im = self.vdo.retrieve()  # retrieve : 획득한 프레임을 실제로 받아오는 함수
            im = cv2.cvtColor(ori_im, cv2.COLOR_BGR2RGB)  
                # BGR -> RGB (opencv는 색을 BGR로 처리하고 matplotlib에서는 RGB로 처리하기 때문에
                # 바꿔줘야 색이 제대로 표시됨)

            # do detection
            bbox_xywh, cls_conf, cls_ids = self.detector(im)  # YOLO v3모델을 이용하여 detection하기 

            # select person class
            mask = cls_ids == 0

            bbox_xywh = bbox_xywh[mask]
            # bbox dilation just in case bbox too small, delete this line if using a better pedestrian detector
            bbox_xywh[:, 3:] *= 1.2
            cls_conf = cls_conf[mask]

            # do tracking
            outputs = self.deepsort.update(bbox_xywh, cls_conf, im)

            # deepsort.update
            #   1. confidence가 min_confidence보다 큰 detection에 대하여 NMS를 적용하고
            #   2. 칼만필터를 이용하여 predict & update 진행
            #       - Update : Matching, Measurement update, Distance Update (현재 active id에 대해서만 dist.)
            #
            #   Return : bbox & id

            # draw boxes for visualization
            if len(outputs) > 0:
                bbox_tlwh = []
                bbox_xyxy = outputs[:, :4]
                identities = outputs[:, -1]
                ori_im = draw_boxes(ori_im, bbox_xyxy, identities)

                for bb_xyxy in bbox_xyxy:
                    bbox_tlwh.append(self.deepsort._xyxy_to_tlwh(bb_xyxy))

                results.append((idx_frame - 1, bbox_tlwh, identities))

            end = time.time()

            if self.args.display:
                cv2.imshow("test", ori_im)
                cv2.waitKey(1)  # 1초가 지나고 다음 라인을 실행

            if self.args.save_path:
                self.writer.write(ori_im)

            # save results
            write_results(self.save_results_path, results, 'mot')

            # logging
            self.logger.info("time: {:.03f}s, fps: {:.03f}, detection numbers: {}, tracking numbers: {}" \
                             .format(end - start, 1 / (end - start), bbox_xywh.shape[0], len(outputs)))


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("VIDEO_PATH", type=str)
    parser.add_argument("--config_detection", type=str, default="./configs/yolov3.yaml")
    parser.add_argument("--config_deepsort", type=str, default="./configs/deep_sort.yaml")
    # parser.add_argument("--ignore_display", dest="display", action="store_false", default=True)
    parser.add_argument("--display", action="store_true")
    parser.add_argument("--frame_interval", type=int, default=1)
    parser.add_argument("--display_width", type=int, default=800)
    parser.add_argument("--display_height", type=int, default=600)
    parser.add_argument("--save_path", type=str, default="./output/")
    parser.add_argument("--cpu", dest="use_cuda", action="store_false", default=True)
    parser.add_argument("--camera", action="store", dest="cam", type=int, default="-1")
    return parser.parse_args()

    '''
    add_argument 인자의 간단한 설명
        action : 인자가 발견될 때의 수행할 액션의 기본형
            - store(default) : 인자 값을 저장

        dest : [parse_args()]가 반환하는 객체에 추가될 attribute의 이름
            - 즉, dest='cam'으로 설정되면 args.cam으로 사용가능
    '''

if __name__ == "__main__":
    args = parse_args()
    cfg = get_config()
    cfg.merge_from_file(args.config_detection)
    cfg.merge_from_file(args.config_deepsort)

    with VideoTracker(cfg, args, video_path=args.VIDEO_PATH) as vdo_trk:
        vdo_trk.run()

### deep_sort folder



#### deep_sort.py

In [ ]:
import numpy as np
import torch

from .deep.feature_extractor import Extractor
from .sort.nn_matching import NearestNeighborDistanceMetric
from .sort.preprocessing import non_max_suppression
from .sort.detection import Detection
from .sort.tracker import Tracker


__all__ = ['DeepSort']


class DeepSort(object):
    def __init__(self, model_path, max_dist=0.2, min_confidence=0.3, nms_max_overlap=1.0, max_iou_distance=0.7, max_age=70, n_init=3, nn_budget=100, use_cuda=True):
        self.min_confidence = min_confidence
        self.nms_max_overlap = nms_max_overlap

        self.extractor = Extractor(model_path, use_cuda=use_cuda)

        max_cosine_distance = max_dist
        nn_budget = 100
        metric = NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
        self.tracker = Tracker(metric, max_iou_distance=max_iou_distance, max_age=max_age, n_init=n_init)

    def update(self, bbox_xywh, confidences, ori_img):
        self.height, self.width = ori_img.shape[:2]
        
        # generate detections
        features = self._get_features(bbox_xywh, ori_img)  
        bbox_tlwh = self._xywh_to_tlwh(bbox_xywh)    
        detections = [Detection(bbox_tlwh[i], conf, features[i]) for i, conf in enumerate(confidences) if conf > self.min_confidence]
            # confidence가 min_confidence보다 큰 detections만 추출

        # detections list에 대하여 NMS(non-maximum supression) 적용
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = non_max_suppression(boxes, self.nms_max_overlap, scores)
        detections = [detections[i] for i in indices]

        # update tracker
        self.tracker.predict()  # kalman filter의 predict부분 (다음 상태 예측값 및 예측공분산 계산)
        self.tracker.update(detections)  # Matching, Measurement update, Distance metrics update


            # tracker.update : 
            #   1) Matching Cascade에 따라 Confirmed된 Track(처음 등장후, n_init이상 프레임에서 detection된 tracker)과 
            #      Detection을 Matching
            #   2) Unconfirmed track 중 time_since_update가 안 된 track과 unmatched detection을 matching
            #   3) Track update(measurement update)
            #   4) distance metrics 업데이트 (현재 프레임에 있는 active target id에 대해서만 계산된 distance metrix로 update)

            # ** 매칭시, cost matrix를 cosine dist. of appearance vector로 정의하였고,
            #    indicator를 이용해 (predicted kalman state와 detection) Mahalanobis dist.가 일정값 이상인경우 또는
            #    cosine dist.가 일정값 이상인 경우를 제외시켜주었음.

        # output bbox and identities (box 좌표와 id를 output)
        outputs = []
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            box = track.to_tlwh()
            x1,y1,x2,y2 = self._tlwh_to_xyxy(box)
            track_id = track.track_id
            outputs.append(np.array([x1,y1,x2,y2,track_id], dtype=np.int))
        if len(outputs) > 0:
            outputs = np.stack(outputs,axis=0)
        return outputs

    """
    TODO:
        Convert bbox from xc_yc_w_h to xtl_ytl_w_h
        즉, (center_x, center_y, w, h) -> (x_topleft, y_topleft, w, h) 변환
    """
    @staticmethod
    def _xywh_to_tlwh(bbox_xywh):
        if isinstance(bbox_xywh, np.ndarray):
            bbox_tlwh = bbox_xywh.copy()
        elif isinstance(bbox_xywh, torch.Tensor):
            bbox_tlwh = bbox_xywh.clone()
        bbox_tlwh[:,0] = bbox_xywh[:,0] - bbox_xywh[:,2]/2.
        bbox_tlwh[:,1] = bbox_xywh[:,1] - bbox_xywh[:,3]/2.
        return bbox_tlwh


    def _xywh_to_xyxy(self, bbox_xywh):
        x,y,w,h = bbox_xywh
        x1 = max(int(x-w/2),0)
        x2 = min(int(x+w/2),self.width-1)
        y1 = max(int(y-h/2),0)
        y2 = min(int(y+h/2),self.height-1)
        return x1,y1,x2,y2

    def _tlwh_to_xyxy(self, bbox_tlwh):
        """
        TODO:
            Convert bbox from xtl_ytl_w_h to xc_yc_w_h
        Thanks JieChen91@github.com for reporting this bug!
        """
        x,y,w,h = bbox_tlwh
        x1 = max(int(x),0)
        x2 = min(int(x+w),self.width-1)
        y1 = max(int(y),0)
        y2 = min(int(y+h),self.height-1)
        return x1,y1,x2,y2

    def _xyxy_to_tlwh(self, bbox_xyxy):
        x1,y1,x2,y2 = bbox_xyxy

        t = x1
        l = y1
        w = int(x2-x1)
        h = int(y2-y1)
        return t,l,w,h
    
    def _get_features(self, bbox_xywh, ori_img):
        im_crops = []
        for box in bbox_xywh:
            x1,y1,x2,y2 = self._xywh_to_xyxy(box)
            im = ori_img[y1:y2,x1:x2]
            im_crops.append(im)
        if im_crops:
            features = self.extractor(im_crops)
        else:
            features = np.array([])
        return features


#### deep folder

##### model.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, c_in, c_out,is_downsample=False):

        super(BasicBlock,self).__init__()
        self.is_downsample = is_downsample
        if is_downsample:
            self.conv1 = nn.Conv2d(c_in, c_out, 3, stride=2, padding=1, bias=False)
        else:
            self.conv1 = nn.Conv2d(c_in, c_out, 3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(c_out)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(c_out,c_out,3,stride=1,padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(c_out)

        if is_downsample:
            self.downsample = nn.Sequential(
                nn.Conv2d(c_in, c_out, 1, stride=2, bias=False),
                nn.BatchNorm2d(c_out)
            )
        elif c_in != c_out:
            self.downsample = nn.Sequential(
                nn.Conv2d(c_in, c_out, 1, stride=1, bias=False),
                nn.BatchNorm2d(c_out)
            )
            self.is_downsample = True

    def forward(self,x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn2(y)
        if self.is_downsample:
            x = self.downsample(x)
        return F.relu(x.add(y),True)

def make_layers(c_in,c_out,repeat_times, is_downsample=False):
    blocks = []
    for i in range(repeat_times):
        if i ==0:
            blocks += [BasicBlock(c_in,c_out, is_downsample=is_downsample),]
        else:
            blocks += [BasicBlock(c_out,c_out),]
    return nn.Sequential(*blocks)

class Net(nn.Module):
    def __init__(self, num_classes=751 ,reid=False):
        super(Net,self).__init__()
        
        # 3 128 64
        self.conv = nn.Sequential(
            nn.Conv2d(3,64,3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # nn.Conv2d(32,32,3,stride=1,padding=1),
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(3,2,padding=1),
        )

        # 32 64 32
        self.layer1 = make_layers(64,64,2,False)
        # 32 64 32
        self.layer2 = make_layers(64,128,2,True)
        # 64 32 16
        self.layer3 = make_layers(128,256,2,True)
        # 128 16 8
        self.layer4 = make_layers(256,512,2,True)
        # 256 8 4
        self.avgpool = nn.AvgPool2d((8,4),1)
        # 256 1 1 
        self.reid = reid
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, num_classes),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        # B x 128
        if self.reid:
            x = x.div(x.norm(p=2,dim=1,keepdim=True))
            return x
        # classifier
        x = self.classifier(x)
        return x


if __name__ == '__main__':
    net = Net()
    x = torch.randn(4,3,128,64)
    y = net(x)
    import ipdb; ipdb.set_trace()


##### feature_extractor.py

In [ ]:
import torch
import torchvision.transforms as transforms
import numpy as np
import cv2
import logging

from .model import Net

class Extractor(object):
    def __init__(self, model_path, use_cuda=True):
        self.net = Net(reid=True)
        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        state_dict = torch.load(model_path, map_location=lambda storage, loc: storage)['net_dict']
        self.net.load_state_dict(state_dict)
        logger = logging.getLogger("root.tracker")
        logger.info("Loading weights from {}... Done!".format(model_path))
        self.net.to(self.device)
        self.size = (64, 128)
        self.norm = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])

    def _preprocess(self, im_crops):
        """
        TODO:
            1. to float with scale from 0 to 1
            2. resize to (64, 128) as Market1501 dataset did
            3. concatenate to a numpy array
            3. to torch Tensor
            4. normalize
        """
        def _resize(im, size):
            return cv2.resize(im.astype(np.float32)/255., size)

        im_batch = torch.cat([self.norm(_resize(im, self.size)).unsqueeze(0) for im in im_crops], dim=0).float()
        return im_batch

    
    def __call__(self, im_crops):
        '''
        Extractor가 호출되었을 때 실행되는 함수
        - 전처리 및 forward 수행
        - pretrained network를 사용하는 것이므로 grad 계산 X (training X)
        '''

        im_batch = self._preprocess(im_crops)
        with torch.no_grad():
            im_batch = im_batch.to(self.device)
            features = self.net(im_batch)
        return features.cpu().numpy()


if __name__ == '__main__':
    img = cv2.imread("demo.jpg")[:,:,(2,1,0)]
    extr = Extractor("checkpoint/ckpt.t7")
    feature = extr(img)
    print(feature.shape)


##### train.py 

In [ ]:
import argparse
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torchvision

from model import Net

parser = argparse.ArgumentParser(description="Train on market1501")
parser.add_argument("--data-dir",default='data',type=str)
parser.add_argument("--no-cuda",action="store_true")
parser.add_argument("--gpu-id",default=0,type=int)
parser.add_argument("--lr",default=0.1, type=float)
parser.add_argument("--interval",'-i',default=20,type=int)
parser.add_argument('--resume', '-r',action='store_true')
args = parser.parse_args()

# device
device = "cuda:{}".format(args.gpu_id) if torch.cuda.is_available() and not args.no_cuda else "cpu"
if torch.cuda.is_available() and not args.no_cuda:
    cudnn.benchmark = True

In [ ]:
# data loading
root = args.data_dir
train_dir = os.path.join(root,"train")
test_dir = os.path.join(root,"test")
transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop((128,64),padding=4),  
       # 128*64 size로 random하게 crop, 상하좌우 4영역에 대하여 padding (default는 0으로 padding됨)
    torchvision.transforms.RandomHorizontalFlip(),  # 좌우반전
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # 주어진 mean, std를 갖는 image를 정규화

])
transform_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize((128,64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
trainloader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(train_dir, transform=transform_train),
    batch_size=64,shuffle=True
)
testloader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(test_dir, transform=transform_test),
    batch_size=64,shuffle=True
)
num_classes = max(len(trainloader.dataset.classes), len(testloader.dataset.classes))

# net definition
start_epoch = 0
net = Net(num_classes=num_classes)
if args.resume:
    assert os.path.isfile("./checkpoint/ckpt.t7"), "Error: no checkpoint file found!"
    print('Loading from checkpoint/ckpt.t7')
    checkpoint = torch.load("./checkpoint/ckpt.t7")
    # import ipdb; ipdb.set_trace()
    net_dict = checkpoint['net_dict']
    net.load_state_dict(net_dict)
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
net.to(device)

# loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), args.lr, momentum=0.9, weight_decay=5e-4)
best_acc = 0.

# train function for each epoch
# appearance vector를 훈련하는 함수
def train(epoch):
    print("\nEpoch : %d"%(epoch+1))
    net.train()
    training_loss = 0.
    train_loss = 0.
    correct = 0
    total = 0
    interval = args.interval
    start = time.time()
    
    for idx, (inputs, labels) in enumerate(trainloader):
        # forward
        inputs,labels = inputs.to(device),labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # accumurating
        training_loss += loss.item()
        train_loss += loss.item()
        correct += outputs.max(dim=1)[1].eq(labels).sum().item()
        total += labels.size(0)

        # print 
        if (idx+1)%interval == 0:
            end = time.time()
            print("[progress:{:.1f}%]time:{:.2f}s Loss:{:.5f} Correct:{}/{} Acc:{:.3f}%".format(
                100.*(idx+1)/len(trainloader), end-start, training_loss/interval, correct, total, 100.*correct/total
            ))
            training_loss = 0.
            start = time.time()
    
    return train_loss/len(trainloader), 1.- correct/total

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0.
    correct = 0
    total = 0
    start = time.time()
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            correct += outputs.max(dim=1)[1].eq(labels).sum().item()
            total += labels.size(0)
        
        print("Testing ...")
        end = time.time()
        print("[progress:{:.1f}%]time:{:.2f}s Loss:{:.5f} Correct:{}/{} Acc:{:.3f}%".format(
                100.*(idx+1)/len(testloader), end-start, test_loss/len(testloader), correct, total, 100.*correct/total
            ))

    # saving checkpoint
    acc = 100.*correct/total
    if acc > best_acc:
        best_acc = acc
        print("Saving parameters to checkpoint/ckpt.t7")
        checkpoint = {
            'net_dict':net.state_dict(),
            'acc':acc,
            'epoch':epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(checkpoint, './checkpoint/ckpt.t7')

    return test_loss/len(testloader), 1.- correct/total

# plot figure
x_epoch = []
record = {'train_loss':[], 'train_err':[], 'test_loss':[], 'test_err':[]}
fig = plt.figure()
ax0 = fig.add_subplot(121, title="loss")
ax1 = fig.add_subplot(122, title="top1err")
def draw_curve(epoch, train_loss, train_err, test_loss, test_err):
    global record
    record['train_loss'].append(train_loss)
    record['train_err'].append(train_err)
    record['test_loss'].append(test_loss)
    record['test_err'].append(test_err)

    x_epoch.append(epoch)
    ax0.plot(x_epoch, record['train_loss'], 'bo-', label='train')
    ax0.plot(x_epoch, record['test_loss'], 'ro-', label='val')
    ax1.plot(x_epoch, record['train_err'], 'bo-', label='train')
    ax1.plot(x_epoch, record['test_err'], 'ro-', label='val')
    if epoch == 0:
        ax0.legend()
        ax1.legend()
    fig.savefig("train.jpg")

# lr decay
def lr_decay():
    global optimizer
    for params in optimizer.param_groups:
        params['lr'] *= 0.1
        lr = params['lr']
        print("Learning rate adjusted to {}".format(lr))

def main():
    for epoch in range(start_epoch, start_epoch+40):
        train_loss, train_err = train(epoch)
        test_loss, test_err = test(epoch)
        draw_curve(epoch, train_loss, train_err, test_loss, test_err)
        if (epoch+1)%20==0:
            lr_decay()


if __name__ == '__main__':
    main()